##### This notebook demonstrates a demo of how you can deploy your first service with Truefoundry.

---


After you complete the guide, you will have a successfully deployed model. Your deployed API will look like this:

![](https://files.readme.io/e1affc0-Screenshot_2022-11-11_at_5.07.48_PM.png)

## Project structure

To complete this guide, you are going to create the following **files**:

- `app.py` : contains our inference and FastAPI code
- `iris_classifier.joblib` : the model file
- `deploy.py`: contains our deployment code
- `requirements.txt` : contains our dependencies

Your **final file structure** is going to look like this:

```
.
├── app.py
├── iris_classifier.joblib
├── deploy.py 
└── requirements.txt
```



As you can see, all the following files are created in the same folder/directory

**Let's create the directory which will contain all this files:-**

In [ ]:
%mkdir develop
%cd develop

/content
/content/develop


# Setup
Let's first setup all the things we need to deploy our service.
* Signup or Login on TrueFoundry
* Setup ServiceFoundry
* Setup Workspace

Let's start with installing servicefoundry.

In [ ]:
pip install -U "servicefoundry"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 103 kB 14.3 MB/s 
     |████████████████████████████████| 78 kB 2.2 MB/s 
     |████████████████████████████████| 62 kB 481 kB/s 
     |████████████████████████████████| 1.1 MB 85.7 MB/s 
     |████████████████████████████████| 57 kB 2.4 MB/s 
     |████████████████████████████████| 54 kB 1.0 MB/s 
     |████████████████████████████████| 59 kB 4.1 MB/s 
     |████████████████████████████████| 182 kB 54.5 MB/s 
     |████████████████████████████████| 96 kB 3.1 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 56 kB 4.3 MB/s 
     |████████████████████████████████| 237 kB 90.8 MB/s 
     |████████████████████████████████| 63 kB 1.9 MB/s 
     |████████████████████████████████| 80 kB 10.2 MB/s 
     |████████████████████████████████

Run the script below to login to servicefoundry

> If your TrueFoundry is hosted anywhere other then "https://app.truefoundry.com" do ensure to change the TRUEFOUNDRY_HOST

In [ ]:
import logging
[logging.root.removeHandler(h) for h in logging.root.handlers]
logging.basicConfig(level=logging.INFO, format='%(asctime)s [%(name)s] %(levelname)-8s %(message)s')

[12/14/22 08:58:19] WARNING  TFY_API_KEY env var is already set. When running   
                             servicefoundry, it will use the api key to         
                             authorize.                                         
                             Login will just save the credentials on disk.      
Already logged in to 'https://app.develop.truefoundry.tech' as 'user-truefoundry' ('user@truefoundry.com')
Please use `sfy login --relogin` or `sfy.login(relogin=True)` to force relogin



In [ ]:
!sfy login --relogin

### Workspace FQN
Once you run the cell below you will get a prompt to enter your workspace. <br>
* Step 1: Click on the link given in the prompt.
* Step 2: Identify the Workspace you want to deploy the application in. 
* Step 3: Copy the Workspace FQN <br>
![Copying Workspace FQN](https://files.readme.io/730fee2-Screenshot_2023-02-28_at_2.08.34_PM.png)
* Step 4: Paste the Workspace FQN in the prompt and press enter.

In [ ]:
# Copy workspace FQN from https://app.truefoundry.com/workspaces
from getpass import getpass
WORKSPACE=getpass("Please enter you WORKSPACE_FQN (https://app.truefoundry.com/workspaces):")

### API KEY
Once you run the cell below you will get a prompt to enter your API KEY. <br>
* Step 1: Click on the link given in the prompt.
* Step 2: Click on Create New API Key button
* Step 3: Enter the name you want to give to the API KEY and then click on generate
* Step 4: Copy the API KEY, You can also download the API KEY for later use <br>
![Copying API KEY](https://files.readme.io/201c8aa-Screenshot_2023-02-28_at_2.28.17_PM.png)
* Step 5: Paste the API KEY in the prompt and press enter.

In [ ]:
TFY_API_KEY = getpass("Please enter your API Key (https://app.truefoundry.com/settings):")

# Step 1: Fetch model

## Model details

For this guide, we have already trained a model.  
The given model has been trained on _iris dataset_.  Then it is stored as a joblib file in [google drive](https://drive.google.com/file/d/1-9nwjs6F7cp_AhAlBAWZHMXG8yb2q_LR/view).

> **Attributes** :  
> sepal length in cm, sepal width in cm, petal length in cm, petal width in cm
>
> **Predicted Attribute** :  
> class of iris plant (one of the following - Iris Setosa, Iris Versicolour, Iris Virginica)

## Download instructions

Download the model from the following [link](https://drive.google.com/file/d/1-9nwjs6F7cp_AhAlBAWZHMXG8yb2q_LR/view).  
Then move the model in your dev directory we created.

Afterwards, your directory should look like this :

```
.
└── iris_classifier.joblib
```

## Step 2: Implement the inference service code.

The first step is to create a web API and deploy the model.  
For this we are going to use [FastAPI](https://fastapi.tiangolo.com/) for this. FastAPI is a modern, intuitive web framework for building web APIs in python.

Create the `app.py` and `requirements.txt` files in the same directory where the model is stored.

```
.
├── iris_classifier.joblib
├── app.py
└── requirements.txt
```

### **`app.py`**

In [ ]:
%%writefile app.py
import os
import joblib
import pandas as pd
from fastapi import FastAPI

model = joblib.load("iris_classifier.joblib")

app = FastAPI()

@app.get("/predict")
def predict(
    sepal_length: float, sepal_width: float, petal_length: float, petal_width: float
):
    data = dict(
        sepal_length=sepal_length,
        sepal_width=sepal_width,
        petal_length=petal_length,
        petal_width=petal_width,
    )
    prediction = int(model.predict(pd.DataFrame([data]))[0])
    return {"prediction": prediction}

Writing app.py


Click on this [link](https://docs.truefoundry.com/recipes/create-a-fastapi-service-code-to-deploy-model) to understand the **`app.py`**:


### **`requirements.txt`**

In [ ]:
%%writefile requirements.txt
fastapi==0.81.0
uvicorn==0.18.3
scikit-learn==1.0.2
joblib
pandas

Writing requirements.txt


## Step 3: Deploying the inference API

You can deploy services on Truefoundry programmatically via our **Python SDK**.

Create a `deploy.py`, after which our file structure will look like this:

**File Structure**

```Text Text
.
├── iris_classifier.joblib
├── app.py
├── deploy.py
└── requirements.txt
```



### **`deploy.py`**

In [ ]:
%%writefile deploy.py
import logging
from servicefoundry import Build, PythonBuild, Service, Resources

logging.basicConfig(level=logging.INFO)

image = Build(
    build_spec=PythonBuild(
        command="uvicorn app:app --port 8000 --host 0.0.0.0",
        requirements_path="requirements.txt"
    )
)

service = Service(
    name="ml-deploy",
    image=image,
    ports=[{"port": 8000}],
    resources=Resources(memory_limit=1500, memory_request=1000),
    env={
        "UVICORN_WEB_CONCURRENCY": "1",
        "ENVIRONMENT": "dev"
    }
)
service.deploy(workspace_fqn=WORKSPACE)

Overwriting deploy.py


Click on this [link](https://docs.truefoundry.com/recipes/deploy-fastapi-service-via-python) to understand the **`deploy.py`**:

Now to deploy our FastAPI Service run the command below

In [ ]:
!python deploy.py

INFO:servicefoundry:Logged in to 'https://app.develop.truefoundry.tech' as 'user-truefoundry' (user@truefoundry.com)
INFO:servicefoundry:Uploading code for service 'fastapi'
INFO:servicefoundry:Archiving contents of dir: '/content/develop'
INFO:servicefoundry:Neither `.tfyignore` file found in /content/develop nor a valid git repository found. We recommend you to create .tfyignore file and add file patterns to ignore
INFO:servicefoundry:Code archive size: '1.88 KiB'
INFO:servicefoundry:🚀 Deployment started for application 'fastapi'. Deployment FQN is 'tfy-dev-cluster:adijha:fastapi:7'.
INFO:servicefoundry:You can find the application on the dashboard:- 'https://app.develop.truefoundry.tech/applications/clbcbrq36003u1ppxftcg52ab?tab=deployments'
